# **StyleGAN2 with Custom Dataset**

### **1. Clone the StyleGAN2 official git repository.**

In [1]:
!head /proc/cpuinfo
print('\n')
!head -n 3 /proc/meminfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 2000.200
cache size	: 39424 KB
physical id	: 0


MemTotal:       26696424 kB
MemFree:        23715752 kB
MemAvailable:   25746124 kB


In [2]:
!nvidia-smi

Thu Sep  9 04:29:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install git+https://github.com/Kaggle/kaggle-api.git --upgrade

  Cloning https://github.com/Kaggle/kaggle-api.git to /tmp/pip-req-build-9vycpj12
  Running command git clone -q https://github.com/Kaggle/kaggle-api.git /tmp/pip-req-build-9vycpj12


In [ ]:
import os
credentials = {"username":"cookysh","key":"8c311e44416df44ba148caaeef725525"}
os.environ['KAGGLE_USERNAME'] = credentials['username']
os.environ['KAGGLE_KEY'] = credentials['key']

In [3]:
#import google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#SKIP this if you already have a stylegan2 folder in your google drive
%cd /content/drive/My\ Drive/
!mkdir stylegan2-colab2
%cd stylegan2-colab2
!git clone https://github.com/dvschultz/stylegan2
%cd stylegan2
!mkdir pkl
%cd pkl
!gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF #inception: https://drive.google.com/open?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
%cd ../
!mkdir results
!mkdir results/00001-pretrained
%cd results/00001-pretrained
!gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
!mv stylegan2-ffhq-config-f.pkl network-snapshot-10000.pkl
%cd ../../
%mkdir datasets

/content/drive/My Drive
/content/drive/My Drive/stylegan2-colab2
Cloning into 'stylegan2'...
remote: Enumerating objects: 552, done.
remote: Total 552 (delta 0), reused 0 (delta 0), pack-reused 552
Receiving objects: 100% (552/552), 22.47 MiB | 18.80 MiB/s, done.
Resolving deltas: 100% (285/285), done.
/content/drive/My Drive/stylegan2-colab2/stylegan2
/content/drive/My Drive/stylegan2-colab2/stylegan2/pkl
Downloading...
From: https://drive.google.com/uc?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
To: /content/drive/My Drive/stylegan2-colab2/stylegan2/pkl/inception_v3_features.pkl
87.3MB [00:00, 155MB/s]
/content/drive/My Drive/stylegan2-colab2/stylegan2
/content/drive/My Drive/stylegan2-colab2/stylegan2/results/00001-pretrained
Downloading...
From: https://drive.google.com/uc?id=1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
To: /content/drive/My Drive/stylegan2-colab2/stylegan2/results/00001-pretrained/stylegan2-ffhq-config-f.pkl
382MB [00:02, 143MB/s]
/content/drive/MyDrive/stylegan2-colab2/stylegan2


In [ ]:
#USE this if you already have a stylegan2 folder in google drive
%cd /content/drive/My\ Drive/stylegan2-colab2/stylegan2
!git pull

/content/drive/My Drive/stylegan2-colab2/stylegan2
Already up to date.


In [4]:
%tensorflow_version 1.x
%cd /content/drive/My\ Drive/stylegan2-colab2/stylegan2

TensorFlow 1.x selected.
/content/drive/My Drive/stylegan2-colab2/stylegan2


### **2.Convert your dataset into .tfrecords files.**

StyleGAN2 requires you to convert your image data into .tfrecords format. 
You can name your own folder after the directory './datasets/your_folder' to save results and set your path where the image data is located.

In [ ]:
!ls /content/drive/MyDrive/3d_reconstruction_sohyeon/ai_art/resized
print(len('/content/drive/MyDrive/3d_reconstruction_sohyeon/ai_art/resized'))

Andy_Warhol_101_512.jpg  Andy_Warhol_29_512.jpg  Andy_Warhol_6_512.jpg
Andy_Warhol_113_512.jpg  Andy_Warhol_35_512.jpg  Andy_Warhol_66_512.jpg
Andy_Warhol_114_512.jpg  Andy_Warhol_36_512.jpg  Andy_Warhol_68_512.jpg
Andy_Warhol_119_512.jpg  Andy_Warhol_37_512.jpg  Andy_Warhol_69_512.jpg
Andy_Warhol_127_512.jpg  Andy_Warhol_38_512.jpg  Andy_Warhol_72_512.jpg
Andy_Warhol_129_512.jpg  Andy_Warhol_39_512.jpg  Andy_Warhol_7_512.jpg
Andy_Warhol_130_512.jpg  Andy_Warhol_40_512.jpg  Andy_Warhol_81_512.jpg
Andy_Warhol_133_512.jpg  Andy_Warhol_41_512.jpg  Andy_Warhol_82_512.jpg
Andy_Warhol_139_512.jpg  Andy_Warhol_44_512.jpg  Andy_Warhol_83_512.jpg
Andy_Warhol_146_512.jpg  Andy_Warhol_4_512.jpg	 Andy_Warhol_8_512.jpg
Andy_Warhol_148_512.jpg  Andy_Warhol_45_512.jpg  Andy_Warhol_86_512.jpg
Andy_Warhol_151_512.jpg  Andy_Warhol_46_512.jpg  Andy_Warhol_88_512.jpg
Andy_Warhol_15_512.jpg	 Andy_Warhol_51_512.jpg  Andy_Warhol_90_512.jpg
Andy_Warhol_158_512.jpg  Andy_Warhol_52_512.jpg  Andy_Warhol_91_512.j

In [ ]:
%pwd

'/content/drive/My Drive/stylegan2-colab2/stylegan2'

In [ ]:
#originally 50 images
!python dataset_tool.py create_from_images ./datasets/ai_art /content/drive/MyDrive/3d_reconstruction_sohyeon/ai_art/resized/

Loading images from "/content/drive/MyDrive/3d_reconstruction_sohyeon/ai_art/resized/"
  0% 0/49 [00:00<?, ?it/s]dataset_tool.py:87: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
100% 49/49 [00:18<00:00,  2.70it/s]


In [ ]:
#newly picked 5 images
!python dataset_tool.py create_from_images ./datasets/ai_art /content/drive/MyDrive/3d_reconstruction_sohyeon/tobetrained

Loading images from "/content/drive/MyDrive/3d_reconstruction_sohyeon/tobetrained"
  0% 0/5 [00:00<?, ?it/s]dataset_tool.py:87: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
100% 5/5 [00:01<00:00,  3.23it/s]


### **3.Training**

For the detailed setting of each parameter, please check on original codes from StyleGAN2 repository.

In [5]:
!nvcc test_nvcc.cu -o test_nvcc -run
import tensorflow as tf

print('Tensorflow version: {}'.format(tf.__version__) )
!nvidia-smi -L
print('GPU Identified at: {}'.format(tf.test.gpu_device_name()))

CPU says hello.
GPU says hello.
Tensorflow version: 1.15.2
GPU 0: Tesla T4 (UUID: GPU-609d7a40-5c7e-8924-8060-66821c2b144a)
GPU Identified at: /device:GPU:0


In [ ]:
!python run_training.py --num-gpus=1 --data-dir=./datasets --config=config-f --dataset=ai_art --total-kimg=500 --mirror-augment=true --metrics=None

Local submit - run_dir: results/00010-stylegan2-ai_art-1gpu-config-f
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
tcmalloc: large alloc 4294967296 bytes == 0x560782594000 @  0x7fd420582001 0x7fd41d06654f 0x7fd41d0b6b58 0x7fd41d0bab17 0x7fd41d159203 0x56077b9e30a4 0x56077b9e2da0 0x56077ba57868 0x56077ba52235 0x56077b9e4fec 0x56077ba25bc9 0x56077ba22ac4 0x56077b9e51f1 0x56077ba54318 0x56077ba51c35 0x56077b923e2c 0x56077ba54318 0x56077ba51c35 0x56077b923e2c 0x56077ba54318 0x56077b9e465a 0x56077ba52b0e 0x56077b9e465a 0x56077ba52d67 0x56077ba51c35 0x56077b923e2c 0x56077ba54318 0x56077ba51c35 0x56077b923e2c 0x56077ba54318 0x56077b9e465a
tcmalloc: large alloc 4294967296 bytes == 0x560882594000 @  0x7fd4205801e7 0x7fd41d06646e 0x7fd41d0b6c7b 0x7fd41d0b735f 0x7fd41d159103 0x56077b9e30a4 0x56077b9e2da0 0x56077ba57868 0x56077ba51c35 0x56077b9e473a 0x56077ba5393b 0x56077ba51c35 0x56077b9e473a 0x56077ba5393b 0x56077b

### **5.Test the model (Generating fake images)**

Generate fake images by testing and zip the files to download.

In [ ]:
!python run_generator.py generate-images --network=/content/stylegan2-colab2/stylegan2results/00010-stylegan2-celebs-1gpu-config-a/network-snapshot-000100.pkl --seeds=3875451-3876000 --truncation-psi=0.5

In [ ]:
!zip -r generated-0.7.zip /content/stylegan2-colab2/stylegan2/results/00010-stylegan2-celebs-1gpu-config-f

### **6.Interpolation and Convert the results into video**

In [ ]:
!python run_generator.py generate-images --network=/content/stylegan2-colab2/stylegan2/results/00010-stylegan2-celebs-1gpu-config-f/network-snapshot-000100.pkl --seeds=3,11,17,25,3 --truncation-psi=0.7

In [ ]:
!ffmpeg -r 24 -i ./results/00000-generate-images/step%05d.png -vcodec libx264 -pix_fmt yuv420p latent-walk-v2.mp4